## Materia: Robotica
## Grupo: 1
## Alumno: Rodriguez Torres Angel Adrian 
## Profesor: M.I. Erik Peña Gonzáles Villela
## Practica 0

# Práctica 0: Nodos y Tópicos

## Objetivo
Que los alumnos comprendan los conceptos básicos relacionados con los Nodos y los Tópicos, mediante la creación de un paquete con políticas en Python.

## Metas
- Creación de un espacio de trabajo de ROS 2.
- Creación de un paquete con políticas de Python.
- Comprender los elementos que contiene el espacio de trabajo al montar un paquete en Python.
- Creación de un paquete que contenga un Nodo publicador y un Nodo Subcriptor-Publicador.
- Programar un nodo publicador que publique una función senoidal que represente RPM y un nodo subscriptor-publicador que transforme la señal en rad/s.

## Previo

### Definición de Nodo y Tópicos en ROS 2

Un **Nodo** es un programa individual encargado de una única tarea específica. Son los bloques de construcción fundamentales de un sistema robótico, donde cada nodo es responsable de una función (ej. controlar motores, leer un sensor). Esta modularidad facilita el desarrollo y la depuración.

Un **Tópico** es un canal de comunicación con nombre que los nodos utilizan para intercambiar mensajes. Un nodo puede **publicar** (enviar) datos en un tópico, y uno o varios nodos pueden **suscribirse** a ese tópico para recibir dichos datos.

### ¿Cómo se documenta la creación de paquetes? (Árbol)

La estructura de un paquete se documenta con un **árbol de directorios**. Es un esquema de texto que muestra la organización jerárquica de las carpetas y archivos del proyecto.

**Ejemplo de un paquete básico de Python en ROS 2:**
```
nombre_del_paquete/
├── package.xml
├── setup.py
└── nombre_del_paquete/
    ├── __init__.py
    └── nombre_del_nodo.py
```

### Los tres tipos de interfaces que maneja ROS 2

1.  **Mensajes (`.msg`):** Para comunicación **unidireccional** (publicar-suscribir). Ideal para flujos continuos de datos como lecturas de sensores.
2.  **Servicios (`.srv`):** Para comunicación **bidireccional** de tipo pregunta-respuesta. Ideal para tareas rápidas que requieren una confirmación inmediata.
3.  **Acciones (`.action`):** Para tareas de **larga duración con retroalimentación**. Ideal para procesos complejos como la navegación, donde se necesita conocer el progreso.

## Desarrollo de la Práctica

El desarrollo consistió en la creación de un paquete de ROS 2 llamado `nodes_topics_py` para establecer un sistema de comunicación que simula la lectura y conversión de la velocidad de un motor.

### 1. Descripción del Sistema de Nodos

El sistema se compone de dos nodos que operan en cadena:

* **Nodo Publicador (`publish_node.py`):** Actúa como un sensor virtual. Su función es generar una **señal senoidal** que representa la velocidad de un motor en revoluciones por minuto (RPM), variando suavemente entre 500 y 1500 RPM. Publica estos datos en el tópico `/publish_topic`.

* **Nodo Suscriptor-Publicador (`sub_node.py`):** Actúa como una unidad de control o conversión. Se suscribe al tópico `/publish_topic` para recibir los valores de RPM. Cada vez que recibe un dato, aplica la fórmula matemática de conversión a radianes por segundo ($rad/s = RPM \times \frac{2 \pi}{60}$) y publica el resultado en un nuevo tópico llamado `/rad_vel_topic`.

### 2. Código Fuente de los Nodos

A continuación se muestra el código final implementado para cada nodo.

**`publish_node.py` (Publicador Senoidal)**

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64
import math

class SineWavePublisher(Node):
    def __init__(self):
        super().__init__('sine_wave_publisher')
        self.publisher_ = self.create_publisher(Float64, 'publish_topic', 10)
        
        self.amplitude = 500.0
        self.offset = 1000.0
        self.frequency = 0.2
        self.time = 0.0
        
        timer_period = 0.1
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.get_logger().info('Nodo publicador senoidal de RPM iniciado.')
        self.get_logger().info('Publicando RPM entre 500 y 1500.')

    def timer_callback(self):
        msg = Float64()
        
        rpm_value = self.amplitude * math.sin(2 * math.pi * self.frequency * self.time) + self.offset
        msg.data = rpm_value
        
        self.publisher_.publish(msg)
        self.get_logger().info(f'Publicando RPM: {msg.data:.2f}')
        
        self.time += 0.1

def main(args=None):
    rclpy.init(args=args)
    sine_publisher = SineWavePublisher()
    rclpy.spin(sine_publisher)
    sine_publisher.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()

**`sub_node.py` (Conversor a rad/s)**

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import Float64
import math

class SpeedConverter(Node):
    def __init__(self):
        super().__init__('speed_converter')
        self.subscription = self.create_subscription(
            Float64,
            'publish_topic',
            self.rpm_callback,
            10)
        self.publisher_ = self.create_publisher(Float64, 'rad_vel_topic', 10)
        self.get_logger().info('Nodo conversor de RPM a Rad/s iniciado.')

    def rpm_callback(self, msg):
        rpm_value = msg.data
        
        rad_s_value = rpm_value * (2 * math.pi / 60.0)

        new_msg = Float64()
        new_msg.data = rad_s_value
        self.publisher_.publish(new_msg)
        self.get_logger().info(f'Recibido RPM: {rpm_value:.2f} -> Publicando Rad/s: {rad_s_value:.2f}')

def main(args=None):
    rclpy.init(args=args)
    speed_converter = SpeedConverter()
    rclpy.spin(speed_converter)
    speed_converter.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()

### 3. Proceso de Construcción y Ejecución
Para compilar y ejecutar el sistema, se siguieron los siguientes pasos desde la terminal:
1.  **Construcción:** Se compiló el paquete desde la raíz del espacio de trabajo (`~/ROS2Dev`) usando `colcon build`.
2.  **Ejecución:** Se abrieron múltiples terminales. En cada una, se activó el entorno con `source install/setup.bash`. Posteriormente, se ejecutaron los dos nodos y una herramienta de verificación.

## Resultados

Para verificar el correcto funcionamiento del sistema, se ejecutaron los dos nodos simultáneamente y se utilizó la herramienta `ros2 topic echo` para observar la salida final.

* **Terminal 1 (Publicador):** Se ejecutó `ros2 run nodes_topics_py publish_node_py`. La terminal mostró mensajes de `Publicando RPM: ...` con valores fluctuando entre 500 y 1500.

* **Terminal 2 (Suscriptor):** Se ejecutó `ros2 run nodes_topics_py subscriber_node_py`. La terminal mostró mensajes de `Recibido RPM: ... -> Publicando Rad/s: ...`, confirmando la recepción y el procesamiento.

* **Terminal 3 (Verificación):** Se ejecutó `ros2 topic echo /rad_vel_topic`. La salida fue un flujo continuo de datos que oscilaba, como se muestra a continuación:

```
data: 91.698
---
data: 98.157
---
data: 104.719
---
data: 111.282
...
data: 155.118
---
data: 156.557
---
data: 157.063
---
data: 156.551
...
```

### Análisis de Resultados
Los valores observados en el tópico `/rad_vel_topic` corresponden exactamente con la conversión esperada. El nodo publicador genera RPM en el rango de **[500, 1500]**. Al aplicar el factor de conversión ($ \frac{2 \pi}{60} \approx 0.1047 $), el rango de salida esperado en rad/s es:

-   **Mínimo:** $500 \times 0.1047 \approx 52.4$ rad/s
-   **Máximo:** $1500 \times 0.1047 \approx 157.1$ rad/s

El rango observado en la terminal se alinea perfectamente con el rango teórico, confirmando el éxito de la implementación.

## Conclusiones

A través de esta práctica, se cumplieron exitosamente todos los objetivos planteados. Se logró crear un espacio de trabajo y un paquete funcional en ROS 2, lo que permitió una comprensión práctica de su estructura y componentes clave como `setup.py` y `package.xml`.

Se implementó de manera efectiva el patrón de comunicación publicador-suscriptor, demostrando el flujo de datos a través de los tópicos. La modificación final para generar una señal senoidal y convertirla de RPM a rad/s consolidó el entendimiento sobre cómo los nodos pueden ser utilizados no solo para transmitir información, sino también para procesarla en tiempo real, una tarea fundamental en robótica. Finalmente, el uso de herramientas como `colcon`, `ros2 run` y `ros2 topic list/echo` demostró ser indispensable para el ciclo de desarrollo, depuración y verificación de un sistema robótico.

## Bibliografía
[1] Open Robotics. (2022). Understanding ROS 2 nodes. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Nodes/Understanding-ROS2-Nodes.html

[2] Open Robotics. (2022). Understanding ROS 2 topics. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Topics/Understanding-ROS2-Topics.html

[3] Open Robotics. (2022). Understanding ROS 2 services. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Services/Understanding-ROS2-Services.html

[4] Open Robotics. (2022). Understanding ROS 2 actions. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-CLI-Tools/Understanding-ROS2-Actions/Understanding-ROS2-Actions.html

[5] Open Robotics. (2022). Creating a package. ROS 2 Documentation: Humble. Disponible en: https://docs.ros.org/en/humble/Tutorials/Beginner-Client-Libraries/Creating-A-Package/Creating-A-Package.html